## 多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

# 2 是批量大小，20 是输入数据的维度
X = torch.rand(2,20)
net(X)

tensor([[ 0.1506, -0.1721,  0.0328,  0.0273,  0.0705,  0.0432, -0.2675,  0.0705,
          0.0802, -0.0505],
        [ 0.1168, -0.2236, -0.0185,  0.1986,  0.0265,  0.0079, -0.2239,  0.0575,
          0.0282, -0.1293]], grad_fn=<AddmmBackward0>)

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(10,256)
        self.out = nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))
        

In [11]:
X = torch.rand(3,10)
net = MLP()
net(X)

tensor([[ 0.1399,  0.1329,  0.0407, -0.3097, -0.1972, -0.1405, -0.1174,  0.1585,
         -0.0074, -0.1313],
        [ 0.0587,  0.3882, -0.0097, -0.2372, -0.1299, -0.1267, -0.0188,  0.1376,
         -0.0616, -0.0951],
        [ 0.2029,  0.1268,  0.0130, -0.2710, -0.2110, -0.1919, -0.1457,  0.0541,
          0.0454, -0.0837]], grad_fn=<AddmmBackward0>)

In [19]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        """顺序执行所有类"""
        print(self._modules)
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(10,256),nn.ReLU(),nn.Linear(256,10))
net(X)


OrderedDict([(Linear(in_features=10, out_features=256, bias=True), Linear(in_features=10, out_features=256, bias=True)), (ReLU(), ReLU()), (Linear(in_features=256, out_features=10, bias=True), Linear(in_features=256, out_features=10, bias=True))])


tensor([[ 0.2133,  0.0457, -0.1385,  0.0625, -0.1255, -0.0967,  0.1359,  0.1135,
          0.0552, -0.1662],
        [ 0.1382,  0.0887,  0.0256,  0.2528, -0.0623, -0.0800,  0.1204,  0.1802,
          0.1139, -0.1001],
        [ 0.2742,  0.1356, -0.1193,  0.0438, -0.1259, -0.1257,  0.1254,  0.0872,
          0.0968, -0.1817]], grad_fn=<AddmmBackward0>)

灵活的前向计算：

In [32]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
            
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
       
        while X.abs().sum() > 1:
            X /= 2 
            print(X)
            print(X.sum())
        return X.sum()

X = torch.rand(2,20)            
net = FixedHiddenMLP()
net(X)

tensor([[-0.1688, -0.0479,  0.3440, -0.3189,  0.1951,  0.1411, -0.0314,  0.2381,
          0.1274,  0.3521,  0.2575, -0.5537, -0.4296,  0.1636, -0.0822, -0.0152,
         -0.4712,  0.3673, -0.3553,  0.1399],
        [-0.2046, -0.1297,  0.4285, -0.2998,  0.2628,  0.1910, -0.2557,  0.2244,
          0.2220,  0.6431,  0.1826, -0.7076, -0.4830,  0.4011,  0.0918,  0.0085,
         -0.7604,  0.4925, -0.5370,  0.2585]], grad_fn=<DivBackward0>)
tensor(-0.1191, grad_fn=<SumBackward0>)
tensor([[-0.0844, -0.0239,  0.1720, -0.1595,  0.0976,  0.0705, -0.0157,  0.1191,
          0.0637,  0.1760,  0.1287, -0.2768, -0.2148,  0.0818, -0.0411, -0.0076,
         -0.2356,  0.1837, -0.1776,  0.0699],
        [-0.1023, -0.0648,  0.2142, -0.1499,  0.1314,  0.0955, -0.1279,  0.1122,
          0.1110,  0.3215,  0.0913, -0.3538, -0.2415,  0.2005,  0.0459,  0.0043,
         -0.3802,  0.2463, -0.2685,  0.1292]], grad_fn=<DivBackward0>)
tensor(-0.0596, grad_fn=<SumBackward0>)
tensor([[-0.0422, -0.0120,  0.0860, -0

tensor(-0.0074, grad_fn=<SumBackward0>)

混合搭配各种组合块：

In [33]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor([[ 0.1415,  0.0149,  0.0335,  0.0573, -0.4084, -0.2105, -0.0499, -0.2289,
         -0.0579,  0.2685,  0.0811,  0.0997, -0.3233,  0.2136,  0.0781,  0.2567,
         -0.2134, -0.2654, -0.0771,  0.1084],
        [ 0.1423,  0.0179,  0.0307,  0.0587, -0.4003, -0.2135, -0.0554, -0.2223,
         -0.0542,  0.2639,  0.0856,  0.0947, -0.3176,  0.2073,  0.0708,  0.2503,
         -0.2089, -0.2603, -0.0847,  0.1018]], grad_fn=<DivBackward0>)
tensor(-0.9745, grad_fn=<SumBackward0>)
tensor([[ 0.0708,  0.0074,  0.0167,  0.0287, -0.2042, -0.1053, -0.0249, -0.1145,
         -0.0289,  0.1343,  0.0405,  0.0498, -0.1616,  0.1068,  0.0391,  0.1284,
         -0.1067, -0.1327, -0.0386,  0.0542],
        [ 0.0711,  0.0089,  0.0154,  0.0293, -0.2001, -0.1067, -0.0277, -0.1111,
         -0.0271,  0.1319,  0.0428,  0.0474, -0.1588,  0.1037,  0.0354,  0.1252,
         -0.1044, -0.1302, -0.0423,  0.0509]], grad_fn=<DivBackward0>)
tensor(-0.4872, grad_fn=<SumBackward0>)
tensor([[ 0.0354,  0.0037,  0.0084,  0

tensor(-0.1218, grad_fn=<SumBackward0>)

## 参数管理

In [35]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

# 2 是批量大小，20 是输入数据的维度
X = torch.rand(2,4)
net(X)

tensor([[0.0644],
        [0.1803]], grad_fn=<AddmmBackward0>)

In [36]:
# 查看标号为 2 的 net 的状态：weights ， bias
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0269,  0.2031,  0.2608, -0.3484,  0.2018, -0.1246,  0.3527, -0.3169]])), ('bias', tensor([0.2592]))])


In [40]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)
print(net[2].weight.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2592], requires_grad=True)
tensor([0.2592])
None
None


In [43]:
# 一次性访问所有参数
print(*[(name,param.shape) for name, param in net[0].named_parameters()])
print(*[(name,param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [44]:
net.state_dict()['2.bias']

tensor([0.2592])

In [45]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.Tanh())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net 

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)


tensor([[-0.4167],
        [-0.4170]], grad_fn=<AddmmBackward0>)

In [46]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): Tanh()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): Tanh()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): Tanh()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): Tanh()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 内置参数初始化

In [3]:
def init_normal(m):
    if type(m) == nn.Linear:
        # 下划线表示替换函数，不返回值。 weight 不能全部初始化成常数
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
# 遍历整个神经网络，做 init_normal 操作
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([-0.0070,  0.0074, -0.0188,  0.0182,  0.0082,  0.0157,  0.0082,  0.0011,
          0.0055, -0.0058, -0.0197,  0.0009, -0.0170, -0.0015,  0.0133, -0.0019,
          0.0040,  0.0055, -0.0075, -0.0030]),
 tensor(0.))

In [7]:
def xavier(m):
    # 优秀的初始化应该使得各层的激活值和状态梯度的方差在传播过程中的方差保持一致
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0921, -0.0661, -0.1078,  0.0974, -0.1087,  0.0505, -0.0241, -0.1206,
        -0.0129,  0.1390, -0.1018,  0.0274,  0.0128,  0.1253,  0.1404,  0.0556,
        -0.0795,  0.0774, -0.0122,  0.0316])
tensor([[42., 42., 42.,  ..., 42., 42., 42.],
        [42., 42., 42.,  ..., 42., 42., 42.],
        [42., 42., 42.,  ..., 42., 42., 42.],
        ...,
        [42., 42., 42.,  ..., 42., 42., 42.],
        [42., 42., 42.,  ..., 42., 42., 42.],
        [42., 42., 42.,  ..., 42., 42., 42.]])


### 自定义初始化

In [8]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([256, 20])
Init weight torch.Size([10, 256])


tensor([[ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  8.7513,
          5.5534, -0.0000, -0.0000, -8.8039, -8.4391, -0.0000,  7.5690,  0.0000,
         -0.0000, -0.0000, -5.9012, -8.3093],
        [-0.0000,  6.6263,  0.0000,  6.5707, -0.0000, -9.2991,  0.0000,  0.0000,
         -0.0000, -9.8854,  6.7244, -5.1630,  0.0000,  8.2028,  8.9095,  0.0000,
          6.7853, -0.0000,  0.0000,  6.2909]], grad_fn=<SliceBackward0>)

In [9]:
net[0].weight.data[:] += 1 # 可以直接做运算
net[0].weight.data[0,0] = 42 # 也可以做替换
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  9.7513,
         6.5534,  1.0000,  1.0000, -7.8039, -7.4391,  1.0000,  8.5690,  1.0000,
         1.0000,  1.0000, -4.9012, -7.3093])

### 参数绑定

In [13]:
X = torch.rand(3,4)
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), shared, 
                    nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
# shared 层的所有参数都是相同的
print(net[2].weight.data[0] == net[4].weight.data[0])
# 一层修改，其他位置的 shared 层共享修改
net[2].weight.data[0, 0] = 100 
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

构造一个没有任何参数的自定义层

In [16]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean() 

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

可以将自定义的层作为组件合并到构建更复杂的模型中

In [23]:
net = nn.Sequential(nn.Linear(8, 10), CenteredLayer())

Y = net(torch.rand(4,8))
Y

tensor([[-0.4130,  0.5728, -0.3892, -0.0608,  0.5987, -0.4191,  0.4672, -0.4810,
          1.0546, -0.7164],
        [-0.1821,  0.2579, -0.4987,  0.2238,  0.1947, -0.3534,  0.1362, -0.2239,
          0.6122, -0.3855],
        [-0.6761,  0.6874, -0.1975, -0.0564,  0.3003, -0.3501,  0.6639, -0.6144,
          0.8944, -0.5816],
        [-0.6511,  0.3101, -0.3287,  0.0239,  0.4241, -0.8036,  0.7564, -0.4005,
          1.2376, -0.6335]], grad_fn=<SubBackward0>)

## 读取文件

加载和保存张量

In [25]:
import torch 
from torch import nn
from torch.nn import functional as F

In [27]:
x = torch.arange(4)
# save tensor
torch.save(x, 'x-file')
# load tensor
x2 = torch.load("x-file")
x2 

tensor([0, 1, 2, 3])

存储一个张量列表，然后把他们读回内存

In [28]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [31]:
mydict = {'x':x, 'y':y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数：**只需要存模型每层的参数**

In [34]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(10,256)
        self.out = nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

将模型的参数存储为一个叫做 "mlp.params" 的文件

In [36]:
net = MLP()
X = torch.rand(2,10)
# net.forward(X)
Y = net(X)

In [37]:
torch.save(net.state_dict(), 'mlp.params')

实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数

In [38]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
# 保证 BN 层能够用全部训练数据的均值和方差，即测试过程中要保证 BN 层的均值和方差不变。
clone.eval() 

MLP(
  (hidden): Linear(in_features=10, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

In [39]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])